# Car Evaluation Dataset

In [1]:
# !pip install ucimlrepo

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Dense

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

from ucimlrepo import fetch_ucirepo 

### Importando o dataset

In [3]:
# fetch dataset 
car_evaluation = fetch_ucirepo(id=19) 
  
# data (as pandas dataframes) 
X = car_evaluation.data.features 
Y = car_evaluation.data.targets 

# Transform labels to int
labels = Y["class"].unique()
for i in range(len(labels)):
  Y.loc[Y['class']==labels[i], 'class'] = i


C:\Users\AMD RYZEN\AppData\Local\Temp\ipykernel_15276\464027119.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y.loc[Y['class']==labels[i], 'class'] = i


### One-hot Encoding das features

In [4]:
X = pd.get_dummies(X, dtype=int)

### Dividindo os conjuntos de teste e treino

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size= 0.7, random_state = 28)

### One-Hot Enconding dos Targets

In [6]:
y_train = to_categorical(y_train) 
y_true = list(y_test['class'])
y_test = to_categorical(y_test) 

In [7]:
print(y_train[0])

[0. 0. 0. 1.]


### Perceptron

In [8]:
model = tf.keras.Sequential([
    Flatten(input_shape=(len(X.columns), )),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(y_train[0]), activation='softmax')
])

In [9]:
model.compile(
  loss='categorical_crossentropy', 
  optimizer='sgd', 
  metrics=['Accuracy', 'Precision', 'Recall', 'F1Score']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 21)                0         
                                                                 
 dense (Dense)               (None, 128)               2816      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 4)                 260       
                                                                 
Total params: 11332 (44.27 KB)
Trainable params: 11332 (44.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Treinamento

In [10]:
model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=1)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/100
17/17 [==============================] - 1s 1ms/step - loss: 1.3120 - Accuracy: 0.4170 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.2043
Epoch 2/100
17/17 [==============================] - 0s 1ms/step - loss: 1.1554 - Accuracy: 0.6834 - precision: 1.0000 - recall: 0.0019 - f1_score: 0.2030    
Epoch 3/100
17/17 [==============================] - 0s 1ms/step - loss: 1.0407 - Accuracy: 0.6931 - precision: 0.8364 - recall: 0.0888 - f1_score: 0.2047
Epoch 4/100
17/17 [==============================] - 0s 1ms/step - loss: 0.9626 - Accuracy: 0.6931 - precision: 0.7333 - recall: 0.4247 - f1_score: 0.2047
Epoch 5/100
17/17 [==============================] - 0s 1ms/step - loss: 0.9118 - Accuracy: 0.6931 - precision: 0.7067 - recall: 0.6699 - f1_score: 0.2047
Epoch 6/100
17/17 [==============================] - 0s 1ms/step - loss: 0.8744 - Accuracy: 0.6931 - precision: 0.6944 - recall: 0.6931 - f1_score: 0.2047
Epoch 7/100
17/17 [==============================] - 0s 1m

[0.27170583605766296,
 0.8917355537414551,
 0.9251337051391602,
 0.8578512668609619,
 array([0.96816033, 0.8033614 , 0.04166666, 0.44444442], dtype=float32)]

In [11]:
predictions = model.predict(x_test)
predictions = [list(p).index(max(p)) for p in predictions]
# print(predictions)
# print(y_true)

print(f'Acurácia obtida: {accuracy_score(y_true, predictions) * 100:.2f}%')
print(f'Precisão obtida: {precision_score(y_true, predictions, average="weighted") * 100:.2f}%')
print(f'Recall obtido: {recall_score(y_true, predictions, average="weighted") * 100:.2f}%')
print(f'F1 Score obtida: {f1_score(y_true, predictions, average="weighted") * 100:.2f}%')


38/38 [==============================] - 0s 622us/step
[1, 0, 0, 1, 1, 3, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 3, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 3, 3, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,